# Introduction

Some example code showing how to automatically download files from a Google drive folder, then load all csv and excel into a Postgres database. The code will handle csv encoding issues as well as load all tabs in each Excel file. 

However, it is only an example and would need more work because:

- Some Excel files have more than one table on a sheet
- Many Excel files have blank rows, you'd need to scan through them with openpyxl and remove so column heading are identified
- Table names are cleaned <FILE NAME>_<SHEET NAME IF EXCEL WITH MULTIPLE SHEETS>

The examples chosen are for the Save the Children stream of the DataKind 2022 DataDive.

- [project brief](https://docs.google.com/document/d/1TQ2TiGK_k8KEIUPzVb3ZSKaxzEBP9s9ZJBI2U_HeQ6U/edit#)
- [Google drive with data](https://drive.google.com/drive/folders/1G_CAhpb0xV9zRrV-T5ngA03x-IZ4tGKz)

The example is meant to show how we might load all data into a database to see how we might link data together, build classification and persist meta data. There are many other ways of course, but good 'ol SQL might be a good start!

## Setup

1. Go to the DataDive [google folder](https://drive.google.com/drive/folders/1eBQZhSeCXC9a62Dpg8_cWeKIZXfrxfGz)
2. Right-click on a sub-folder and Create a shortcut in your My Drive, eg for Kenya folder
3. Run pane to mount google drive below, connect your drive
5. Edit `dir` variable below and set path as seen in the laft pane

In [ ]:
# If you don't have these installed, uncomment these lines and run
#%pip install gdown
#%pip install chardet
#%pip install sqlalchemy
#%pip install psycopg2-binary

In [ ]:
import gdown
import os
import pandas as pd
import chardet

## Examples
### Downloading a folder from gdrive

In [17]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
#folder_url = "https://drive.google.com/drive/folders/1G_CAhpb0xV9zRrV-T5ngA03x-IZ4tGKz?usp=share_link"
#gdown.download_folder(folder_url, quiet=True, use_cookies=False)

Mounted at /content/drive


## Reading files in a directory to pandas

This snippet will read all csv and xls files in a directory into pandas dataframes, create a list of them.

Just an example, needs a bit more logic to handle xlsx sheets which hold multiple tables, blank rows etc.

In [60]:
# TODO - Make book scan all folder
country = "Somalia"
dir = f"/content/drive/MyDrive/{country}"

#dir = "/content/drive/MyDrive/Kenya Cleaned Up Files"


meta_data = pd.read_csv(f"/content/drive/MyDrive/Kenya/Data metadata - Sheet1.csv")
file_col = 'Google Drive or Github link to downloaded data (if you can put what you’ve downloaded into a shared location!)'

# Array of pandas dataframes
dfs = {}

for f in os.listdir(dir):
    file = f
    f = f"{dir}/{f}"
    if '.csv' in f:
        print(f"Loading csv file {f}")
        # Detect encoding
        with open(f, 'rb') as rawdata:
            r = chardet.detect(rawdata.read(100000))
        df = pd.read_csv(f, encoding=r['encoding'])
        df['file'] = file
        df['sheet'] = ''
        dfs[f] = df
    if '.xlsx' in f:
        print(f"Leading {f} ...")
        # Read all sheets in
        sheet_to_df_map = pd.read_excel(f, sheet_name=None)
        for sheet in sheet_to_df_map:
          # TODO
          # 1. TEST FOR BLANK ROWS AT TOP, AND REMOVE
          # 2. Does the table have a blank column, if so, is it two tables on one sheet. If so handle that
          df = sheet_to_df_map[sheet]
          df['file'] = file
          df['sheet'] = sheet
          dfs[f"{f} - {sheet}"] = df 


Leading /content/drive/MyDrive/Somalia/Somalia - Who is Doing What and Where (3W)- October 2022.xlsx ...
Loading csv file /content/drive/MyDrive/Somalia/wfp_food_prices_som.csv
Loading csv file /content/drive/MyDrive/Somalia/Somalia Monthly FEWS NET Staple Food Price.csv
Leading /content/drive/MyDrive/Somalia/Somalia_Operational Presence_March.xlsx ...
Leading /content/drive/MyDrive/Somalia/Somalia - Health Facilities.xlsx ...
Leading /content/drive/MyDrive/Somalia/Somalia Violent and Threatening Incidents Against Healthcare_2020.xlsx ...
Leading /content/drive/MyDrive/Somalia/Somalia - IPC Analysis 2017-2022.xlsx ...
Leading /content/drive/MyDrive/Somalia/2021-post-gu-amn-burden-and-prevalence-9-sep-2021.xlsx ...
Leading /content/drive/MyDrive/Somalia/population-15.7_final.xlsx ...
Leading /content/drive/MyDrive/Somalia/somalia-population-statistics.xlsx ...
Loading csv file /content/drive/MyDrive/Somalia/som_pplp_adm2_v2.csv
Leading /content/drive/MyDrive/Somalia/Somalia-Humanitarian

In [63]:
# TODO Save dfs to csv, as the started for the meta data review, then put on the google for team to edit
for key in dfs:
    print("\n\n===================================================================================================\n\n")
    df = dfs[key]
    f = df['file'][0]
    print(f"File + Sheet: {key} > Size: {df.shape}\n\n")
    file_meta = meta_data.loc[meta_data[file_col] == f]
    #description = file_meta['Dataset description'][0]
    #link = file_meta['Dataset'][0]
    # print("Link: ", str(link),"\n\n")
    # print("Description: \n\n",str(description),"\n\n")
    # TODO Output to a meta data csv file, columns: file_meta, sheet name, df.column, df.types
    display(df)


print("Done")





File + Sheet: /content/drive/MyDrive/Somalia/Somalia - Who is Doing What and Where (3W)- October 2022.xlsx - 3W_Data > Size: (44473, 8)




,Cluster,Org. Name,Org. Acronym,Org Type,Region,District,file,sheet
0,#sector +cluster +code,#org +name,#org +acronym,#org +type +name,#adm1+name,#adm2+name,Somalia - Who is Doing What and Where (3W)- Oc...,3W_Data
1,CCCM,Agency for Technical Cooperation and Development,ACTED,INGO,Bay,Baydhaba,Somalia - Who is Doing What and Where (3W)- Oc...,3W_Data
2,CCCM,Agency for Technical Cooperation and Development,ACTED,INGO,Woqooyi Galbeed,Hargeysa,Somalia - Who is Doing What and Where (3W)- Oc...,3W_Data
3,CCCM,Agency for Technical Cooperation and Development,ACTED,INGO,Sanaag,Ceel Afweyn,Somalia - Who is Doing What and Where (3W)- Oc...,3W_Data
4,CCCM,Agency for Technical Cooperation and Development,ACTED,INGO,Sanaag,Ceerigaabo,Somalia - Who is Doing What and Where (3W)- Oc...,3W_Data
...,...,...,...,...,...,...,...,...
44468,Nutrition,SOS Children's Village,SOS,INGO,Bay,Buur Hakaba,Somalia - Who is Doing What and Where (3W)- Oc...,3W_Data
44469,Nutrition,SOS Children's Village,SOS,INGO,Bay,Buur Hakaba,Somalia - Who is Doing What and Where (3W)- Oc...,3W_Data
44470,Nutrition,SOS Children's Village,SOS,INGO,Bay,Buur Hakaba,Somalia - Who is Doing What and Where (3W)- Oc...,3W_Data
44471,Nutrition,SOS Children's Village,SOS,INGO,Bay,Buur Hakaba,Somalia - Who is Doing What and Where (3W)- Oc...,3W_Data






File + Sheet: /content/drive/MyDrive/Somalia/wfp_food_prices_som.csv > Size: (21860, 16)




,date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice,file,sheet
0,#date,#adm1+name,#adm2+name,#loc+market+name,#geo+lat,#geo+lon,#item+type,#item+name,#item+unit,#item+price+flag,#item+price+type,#currency,#value,#value+usd,wfp_food_prices_som.csv,
1,1995-01-15,Banadir,Banadir,Mogadishu,2.0666667,45.3666667,cereals and tubers,Sorghum (red),KG,actual,Retail,SOS,700.0,0.085,wfp_food_prices_som.csv,
2,1995-02-15,Banadir,Banadir,Mogadishu,2.0666667,45.3666667,cereals and tubers,Sorghum (red),KG,actual,Retail,SOS,525.0,0.0628,wfp_food_prices_som.csv,
3,1995-03-15,Banadir,Banadir,Mogadishu,2.0666667,45.3666667,cereals and tubers,Sorghum (red),KG,actual,Retail,SOS,600.0,0.0708,wfp_food_prices_som.csv,
4,1995-04-15,Banadir,Banadir,Mogadishu,2.0666667,45.3666667,cereals and tubers,Sorghum (red),KG,actual,Retail,SOS,900.0,0.1046,wfp_food_prices_som.csv,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21855,2022-07-15,Togdheer,Burco,Burco,9.52749,45.537472,miscellaneous food,Sugar (white),KG,forecast,Retail,SLS,0.0,NaN,wfp_food_prices_som.csv,
21856,2022-07-15,Togdheer,Burco,Burco,9.52749,45.537472,non-food,Exchange rate,USD/LCU,forecast,Retail,SLS,0.0,NaN,wfp_food_prices_som.csv,
21857,2022-07-15,Togdheer,Burco,Burco,9.52749,45.537472,oil and fats,"Oil (vegetable, imported)",L,forecast,Retail,SLS,0.0,NaN,wfp_food_prices_som.csv,
21858,2022-07-15,Woqooyi Galbeed,Hargeysa,Hargeysa,9.559749603271484,44.06678009033203,cereals and tubers,Sorghum (white),KG,forecast,Retail,SLS,0.0,NaN,wfp_food_prices_som.csv,






File + Sheet: /content/drive/MyDrive/Somalia/Somalia Monthly FEWS NET Staple Food Price.csv > Size: (33858, 17)




...





File + Sheet: /content/drive/MyDrive/Somalia/somalia-population-statistics.xlsx - Region Population Statistics > Size: (19, 8)




,Admin1 Pcode,Admin1 Name,UNFPA 2014 Total\npopulation,UNFPA 2014 Urban\npopulation,UNFPA 2014 Rural\nPopulation,UNFPA 2014 IDP\nPopulation,file,sheet
0,#adm1 +code,#adm1 +name,#population +all,#population +urban,#population +rural,NaN,somalia-population-statistics.xlsx,Region Population Statistics
1,SO11,Awdal,673264,287822,377452,7990.0,somalia-population-statistics.xlsx,Region Population Statistics
2,SO12,Woqooyi Galbeed,1242003,802740,394673,44590.0,somalia-population-statistics.xlsx,Region Population Statistics
3,SO13,Togdheer,721363,483724,211879,25760.0,somalia-population-statistics.xlsx,Region Population Statistics
4,SO14,Sool,327427,120993,201614,4820.0,somalia-population-statistics.xlsx,Region Population Statistics
5,SO15,Sanaag,544123,159717,383496,910.0,somalia-population-statistics.xlsx,Region Population Statistics
6,SO16,Bari,730147,471784,198717,59646.0,somalia-population-statistics.xlsx,Region Population Statistics
7,SO17,Nugaal,392697,138929,244273,9495.0,somalia-population-statistics.xlsx,Region Population Statistics
8,SO18,Mudug,717862,381493,265487,70882.0,somalia-population-statistics.xlsx,Region Population Statistics
9,SO19,Galgaduud,569434,183553,266113,119768.0,somalia-population-statistics.xlsx,Region Population Statistics






File + Sheet: /content/drive/MyDrive/Somalia/somalia-population-statistics.xlsx - District Population Statistics > Size: (75, 10)




,Admin1 Pcode,Admin Name,Admin2 Pcode,Admin2 Name,UNFPA 2014 Total\npopulation,UNFPA 2014 Urban\npopulation,UNFPA 2014 Rural\nPopulation,UNFPA 2014 IDP\nPopulation,file,sheet
0,#adm1 +code,#adm1 +name,#adm2 +code,#adm2 +name,#population +total,#population +urban,#population +rural,NaN,somalia-population-statistics.xlsx,District Population Statistics
1,SO11,Awdal,SO1101,Borama,398609,271045,127504,60.0,somalia-population-statistics.xlsx,District Population Statistics
2,SO11,Awdal,SO1102,Baki,96885,4243,92642,0.0,somalia-population-statistics.xlsx,District Population Statistics
3,SO11,Awdal,SO1103,Lughaye,100818.999996,6407,86551.999996,7860.0,somalia-population-statistics.xlsx,District Population Statistics
4,SO11,Awdal,SO1104,Zeylac,76951,6127,70754,70.0,somalia-population-statistics.xlsx,District Population Statistics
...,...,...,...,...,...,...,...,...,...,...
70,SO27,Middle Juba,SO2703,Saakow,79116.029538,18006,54110.029538,7000.0,somalia-population-statistics.xlsx,District Population Statistics
71,SO28,Lower Juba,SO2801,Kismaayo,162733,116440,36293,10000.0,somalia-population-statistics.xlsx,District Population Statistics
72,SO28,Lower Juba,SO2802,Afmadow,172485,34783,124702,13000.0,somalia-population-statistics.xlsx,District Population Statistics
73,SO28,Lower Juba,SO2803,Badhaadhe,56178,11483,44095,600.0,somalia-population-statistics.xlsx,District Population Statistics


IndexError: ignored

## Loading pandas dataframes into a postgres database

Roughcode for how one might load all the dataframes into a SQL database. You can install postgres locally, or use docker and run [this](https://hub.docker.com/_/postgres).

In [ ]:
from sqlalchemy import create_engine  
import sqlalchemy

# Set these for your Postgres instance
host   = 'localhost' 
dbname = 'postgres'
user   = 'postgres'
pwd    = os.getenv['PWD'] # Don't save passwords in notebooks - that's naughty! :) - use environment variables. :)
port   = '5432'

engine = create_engine('postgresql+psycopg2://' + user + ':' + pwd + '@' + host +':' + port + '/' + dbname)

for k in dfs:
    df = dfs[k]
    table = k.replace('.','').replace(' ','_').replace('-','_').replace('/','').replace(',','')[0:62]
    print(f"Loading into database table {table} ...")
    df.to_sql(table, engine, if_exists='replace')

print("Done")

Loading into database table Kenyafts_requirements_funding_covid_kencsv ...
Loading into database table Kenyadbo_foodconsumptionscorescsv ...
Loading into database table Kenyaper_capita_food_consumption_2017_and_2018xlsx___Sheet1 ...
Loading into database table Kenyaretail_prices_for_dry_beans_2019_per_kgxlsx___Prices_for_ ...
Loading into database table Kenyavalue_of_recorded_marketed_agricultural_production_at_cur ...
Loading into database table Kenyawfp_food_prices_kencsv ...
Loading into database table Kenyafts_requirements_funding_cluster_kencsv ...
Loading into database table Kenyafts_incoming_funding_kencsv ...
Loading into database table Kenyahealth_kencsv ...
Loading into database table Kenyaken_adminboundaries_tabulardataxlsx___Admin2 ...
Loading into database table Kenyaken_adminboundaries_tabulardataxlsx___Admin1 ...
Loading into database table Kenyaken_adminboundaries_tabulardataxlsx___Admin0 ...
Loading into database table Kenyafts_requirements_funding_kencsv ...
Loading i

## Query from the database

In [ ]:
query = ' SELECT * FROM public."Kenyakenya_3w_final_list_2017xlsx___drr"'
df = pd.read_sql(query, con=engine)
display(df)

,index,Unnamed: 0,Unnamed: 1
0,0,Baringo,WVI
1,1,Embu,Trocaire
2,2,Garissa,ADESO
3,3,Homa Bay,PLAN
4,4,Isiolo,ADESO
5,5,Kilifi,PLAN
6,6,Kisumu,PLAN
7,7,Kitui,Trocaire
8,8,Kwale,PLAN
9,9,Machakos,PLAN


### Select all column names in the files which uploaded ok

In [ ]:
query = ' SELECT * FROM information_schema.columns'
df = pd.read_sql(query, con=engine)
display(df)

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,postgres,public,Kenyakenya_3w_final_list_2017xlsx___Table,Total,14,None,YES,double precision,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
1,postgres,pg_catalog,pg_statistic,stanumbers5,26,None,YES,ARRAY,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2,postgres,pg_catalog,pg_statistic,stavalues1,27,None,YES,anyarray,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
3,postgres,pg_catalog,pg_statistic,stavalues2,28,None,YES,anyarray,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
4,postgres,pg_catalog,pg_statistic,stavalues3,29,None,YES,anyarray,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2843,postgres,information_schema,sql_parts,feature_id,1,None,YES,character varying,NaN,1.073742e+09,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2844,postgres,pg_catalog,pg_partitioned_table,partexprs,8,None,YES,pg_node_tree,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2845,postgres,information_schema,sql_parts,feature_name,2,None,YES,character varying,NaN,1.073742e+09,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2846,postgres,information_schema,sql_parts,is_supported,3,None,YES,character varying,3.0,1.200000e+01,...,NO,None,None,None,None,None,NO,NEVER,None,YES
